# Определение уязвимых групп населения

## Итоговый проект первого года обучения Skillfactory Data Science Pro

**"Техническое задание"**

* кластеризовать регионы России о определить, какие из них наиболее остро нуждаются в помощи малообеспеченным/неблагополучным слоям населения;
* описать группы населения, сталкивающиеся с бедностью;
* определить:
  + влияет ли число детей, пенсионеров и других социально уязвимых групп на уровень бедности в регионе;
  + связаны ли уровень бедности/социального неблагополучия с производством и потреблением в регионе;
  + какие еще зависимости можно наблюдать относительно социально незащищенных слоев населения.

## Часть 1. Компиляция данных

In [1]:
import numpy as np
import pandas as pd
import pickle
import regex as re

Данные компилируются частично из предоставленных в проекте файлов, частично - по дополнительным источникам (указаны в тексте).

Сведения берутся по 85 федеральным субъектам. Архангельская, Тюменская области и автономные округа в их составе рассматриваются по-отдельности. В качестве опорной точки будем ориентироваться на данные 2020 г. - как станет понятно, после считывания файлов, сведения по этому году представлены наиболее полно, и результаты будут более согласованными. В случае необходимости можно рассматривать другие близкие годы или динамику по годам.

Ниже - функция для извлечения данных по федеральным субъектам и стандартизации их названий. 
- df - DataFrame;
- standard_names - словарь стандартных названий; 
- список names_to_drop позволяет сразу исключить некоторые регионы, которые могут быть ошибочно обработаны функцией как федеральные субъекты;
- region_col - колонка DataFrame с названиями регионов.

In [2]:
def federal_subj(
    df, standard_names, names_to_drop=None, region_col='region'):
    """Считывание данных по федеральным субъектам"""
       
    def get_standard_name(x):
        """Функция для .apply()
        Просмотр столбца регионов и вывод названий в стандартном виде"""
    
        for key, value in standard_names.items():
            x = re.sub('[\(].*?[\)]', '', x)  # удаляем всё, что в скобках ()
            if re.search(key, x.lower()):
                return value
        return None
    
    if names_to_drop is None:
        names_to_drop = []
    df = df[~df[region_col].isin(names_to_drop)].copy()
    df[region_col] = df[region_col].apply(get_standard_name)
    return (df.dropna(subset=region_col)
            .set_index(region_col).sort_index()) 

### 1.1. Денежные доходы

| Dataframe | Описание |
| - | - |
| per_capita: | Среднедушевые денежные доходы (в месяц), руб. |
| real_incomes | Реальные денежные доходы, в процентах к предыдущему году (покупательная способность: с учетом роста цен)|
| formal_wage |Среднемесячная номинальная начисленная заработная плата, руб. |
| real_pay | Реальная начисленная заработная плата, в процентах к предыдущему году (покупательная способность: с учетом роста цен)|

Считываем информацию с excel-листов.

In [3]:
def finances_readout(file, sheet):
    "Считывание финансовых данных с excel-листа"
    df = pd.read_excel(file, sheet_name=sheet)
    names_to_drop = ['Архангельская область', 'Тюменская область']
    df = federal_subj(df, standard_names, names_to_drop)
    df.columns = pd.to_datetime((df.columns).astype(int), format='%Y').year
    return df
    

with open('social_russia_data/standard_names.pkl', 'rb') as f:
    standard_names = pickle.load(f)
file = './social_russia_data/cash_real_income_wages_2015_2020.xlsx'
per_capita = finances_readout(file, 0)
real_incomes = finances_readout(file, 1)
formal_wage = finances_readout(file, 2)
real_pay = finances_readout(file, 3)

display('Номинальный доход на душу населения', per_capita.head(3))
display('Реальный доход, %', real_incomes.head(3))
display('Номинальная зарплата', formal_wage.head(3))
display('Реальная зарпалата, %', real_pay.head(3))

'Номинальный доход на душу населения'

,2015,2016,2017,2018,2019,2020
region,,,,,,
Алтайский край,20860.0,21256.0,22139.0,22829.0,23937.0,23864.0
Амурская область,28240.0,27976.0,29213.0,30937.0,33304.0,35499.0
Архангельская область без Ненецкого АО,29716.0,29837.0,30707.0,32054.0,33874.0,34852.0


'Реальный доход, %'

,2015,2016,2017,2018,2019,2020
region,,,,,,
Алтайский край,99.1,94.7,100.0,99.7,99.6,95.5
Амурская область,96.1,92.1,101.1,102.4,101.7,100.3
Архангельская область без Ненецкого АО,95.1,93.0,98.7,101.7,100.2,98.4


'Номинальная зарплата'

,2015,2016,2017,2018,2019,2020
region,,,,,,
Алтайский край,20090.0,21202.0,22743.0,25519.0,27962.0,30072.0
Амурская область,32902.0,33837.0,37368.0,42315.0,47234.0,52430.0
Архангельская область без Ненецкого АО,35592.0,38118.0,40352.0,45427.0,49435.0,52779.0


'Реальная зарпалата, %'

,2015,2016,2017,2018,2019,2020
region,,,,,,
Алтайский край,90.0,98.4,103.6,109.3,104.9,103.8
Амурская область,88.0,96.0,107.4,110.1,106.0,105.2
Архангельская область без Ненецкого АО,92.4,100.0,102.5,110.7,104.0,102.9


### 1.2. Детская смертность

Единица измерения - количество человек. Берем суммарные данные по городской и сельской статистике. Данные по Архангельской и Тюменской областям пересчитываем самостоятельно из-за различающейся организации данных в зависимости от года. 

In [4]:
def mortality_readout(file):
    "Считывание данных с excel-листа"
    df = pd.read_excel(file, header=2, skipfooter=13, usecols='A, D:AI')
    df.rename({'Unnamed: 0': 'region'}, axis=1, inplace=True)
    names_to_drop = [
        '            Архангельская область (кроме Ненецкого автономного округа)',
        '            Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)',
    ]
    df = federal_subj(df, standard_names, names_to_drop)
    df.loc['Архангельская область без Ненецкого АО'] \
        = (df.loc['Архангельская область без Ненецкого АО']
           - df.loc['Ненецкий автономный округ'])
    df.loc['Тюменская область без округов'] \
        = (df.loc['Тюменская область без округов']
           - df.loc['Ханты-Мансийский автономный округ - Югра']
           - df.loc['Ямало-Ненецкий автономный округ'])
    df.columns = pd.to_datetime(df.columns, format='%Y').year
    return df


rural = mortality_readout(
    'social_russia_data/child_mortality_rural_1990_2021.xls')
urban = mortality_readout(
    'social_russia_data/child_mortality_urban_1990_2021.xls')
child_mortality = rural.add(urban)
child_mortality.head(3)

,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
region,,,,,,,,,,,,,,,,,,,,,
Алтайский край,522.0,567.0,529.0,560.0,497.0,490.0,405.0,375.0,340.0,391.0,...,330.0,290.0,275.0,222.0,211.0,185.0,176.0,114.0,97.0,88.0
Амурская область,343.0,293.0,280.0,244.0,242.0,250.0,245.0,268.0,246.0,270.0,...,161.0,119.0,101.0,83.0,53.0,48.0,46.0,43.0,42.0,36.0
Архангельская область без Ненецкого АО,288.0,274.0,245.0,237.0,224.0,206.0,194.0,157.0,167.0,168.0,...,108.0,113.0,97.0,84.0,79.0,72.0,53.0,58.0,29.0,32.0


### 1.3. Инвалидность среди трудоспособного возраста

Единица измерения - количество человек. Таблица содержит данные по месяцам и для разных возрастов, берем данные на 1 января каждого года по непенсионным (до 60 лет) возрастам: количество инвалидов пожилого возраста может зависеть, в том, числе, от естественного старения. Инвалидность среди трудоспособного населения дает более точную оценку социального неблагополучия региона.  

In [5]:
disabled = pd.read_csv(
    'social_russia_data/disabled_total_by_age_2017_2022.csv')
disabled = federal_subj(disabled, standard_names)

disabled['date'] = pd.to_datetime(disabled['date'])
disabled['dayofyear'] = disabled['date'].dt.dayofyear
disabled = disabled[disabled['dayofyear'] == 1]
disabled['young'] = disabled[['18_30', '31_40', '41_50', '51_60']].sum(axis=1)
disabled_18_60 = pd.pivot_table(
    data=disabled, values='young', index='region', columns='date')
disabled_18_60.columns = (disabled_18_60.columns).year
disabled_18_60.head(3)

date,2017,2018,2019,2020,2021,2022
region,,,,,,
Алтайский край,66770.0,64605.0,62840.0,61378.0,59810.0,59006.0
Амурская область,27892.0,26720.0,25976.0,25069.0,24149.0,23356.0
Архангельская область без Ненецкого АО,29228.0,28262.0,27615.0,26984.0,26498.0,26009.0


### 1.4. Алкоголизм, наркомания

Единица измерения - количество акогольных/наркотических психозов на 100 тыс. населения.

In [6]:
def alco_readout(file, sheet):
    "Считывание данных с excel-листа"
    df = pd.read_excel(file, sheet_name=sheet)
    df = federal_subj(df, standard_names)
    df.columns = pd.to_datetime((df.columns).astype(int), format='%Y').year
    return df


file = 'social_russia_data/drug_alco.xlsx'
alco = alco_readout(file, 0)
alco1718 = alco_readout(file, 1)
drug = alco_readout(file, 2)
drug1718 = alco_readout(file, 3)
alco = pd.concat([alco, alco1718], axis=1).sort_index()
drug = pd.concat([drug, drug1718], axis=1).sort_index()

display('Алкоголизм', alco.head(3))
display('Наркомания', drug.head(3))

'Алкоголизм'

,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
region,,,,,,,,,,,,,,
Алтайский край,227.6,202.4,173.9,199.8,177.9,164.0,157.6,138.9,105.0,111.9,115.7,100.1,85.9,88.1
Амурская область,174.3,159.0,120.9,113.3,115.5,140.5,124.7,104.8,100.0,96.7,97.2,92.3,91.4,113.3
Архангельская область без Ненецкого АО,120.1,115.2,102.5,89.8,96.5,99.5,84.4,73.2,77.6,NaN,NaN,NaN,55.3,64.7


'Наркомания'

,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
region,,,,,,,,,,,,,,
Алтайский край,29.8,35.2,35.2,37.7,27.3,24.5,26.4,33.4,22.9,21.6,20.7,17.4,17.4,17.4
Амурская область,21.3,27.2,27.2,28.4,26.2,37.1,33.9,24.2,22.5,22.2,19.1,18.9,18.7,24.4
Архангельская область без Ненецкого АО,1.5,1.4,1.4,2.6,3.4,4.6,6.4,4.0,4.0,NaN,NaN,NaN,5.4,9.2


### 1.5. Валовый региональный продукт на душу населения

Считываем данные с 2011 года - обеспечивается единая методика расчета по Архангельской и Тюменской областям.

In [7]:
grp = pd.read_excel('social_russia_data/gross_regional_product_1996_2020.xls', 
                    header=2, skipfooter=1, usecols='A, Q:Z')
grp.rename({'Unnamed: 0': 'region'}, axis=1, inplace=True)
names_to_drop = [
    '        Архангельская область',
    '        Тюменская область']
grp = federal_subj(grp, standard_names, names_to_drop)
grp.columns = pd.to_datetime(grp.columns).year
grp.head(3)

,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
region,,,,,,,,,,
Алтайский край,137677.2,153556.7,173763.5,186798.6,204933.1,224525.8,231268.4,247599.3,271319.7,291156.9
Амурская область,273175.8,280023.9,258817.0,286282.6,343385.7,370192.4,373935.1,419905.2,521060.1,571362.1
Архангельская область без Ненецкого АО,232540.7,270662.9,283264.5,310817.4,352837.9,400764.6,441961.6,493205.1,509917.0,514200.4


### 1.6. Жилищные условия по данным 2020 г.

Выделяем: 
- среднюю жилую площадь на одного человека;
- процент домохозяйств, указавших на плохое состояние жилого помещения;
- процент домохозяйств, не имеющих средств для улучшения жилищных условий (процент встающих в очередь на жилье или рассчитывающих на переселение в связи со сносом дома)

In [8]:
def housing_readout(file, sheet, cols, skipfooter=0):
    "Считывание данных с excel-листа"
    df = pd.read_excel(
        file, sheet_name=sheet, usecols=cols, skipfooter=skipfooter)
    df.rename({'Регион': 'region'}, axis=1, inplace=True)
    
    # Ошибки в написании названий (латиница?)
    df.replace({'Bладимирская область': 'Владимирская'}, inplace=True)
    df.replace({'Bолгоградская область': 'Волгоградская'}, inplace=True)
    df.replace({'Bологодская область': 'Вологодская'}, inplace=True)
    df.replace({'Bоронежская область': 'Воронежская'}, inplace=True)
    df.replace('...', 0, inplace=True)  # слабая статистика -> 0
    df = federal_subj(df, standard_names)
    return df


file = 'social_russia_data/housing_2020.xlsx'
living_area = housing_readout(file, 0, 'A, H')
living_area.columns.values[0] = 'living area'
intents = housing_readout(file, 1, 'A, C, E, H, I', 2)
intents.columns.values[0] = 'to improve'
intents.columns.values[1] = 'bad conditions'
intents.columns.values[2] = 'queue'
intents.columns.values[3] = 'demolition'
intents['no funds'] = ((intents['to improve']/100 
    * (intents['queue']/100 + intents['demolition']/100)) * 100)
housing = pd.concat(
    [living_area, intents[['bad conditions', 'no funds']]], axis=1)
housing.head(3)

,living area,bad conditions,no funds
region,,,
Алтайский край,18.0,0.3,0.8642
Амурская область,17.3,1.3,1.0731
Архангельская область без Ненецкого АО,16.9,1.4,1.2998


### 1.7. Заболеваемость

Для обеспечения единой методики расчета по Архангельской и Тюменской областям берем данные с 2014 г. (данные рассчитывались в перерасчете на 100 тыс. человек, при этом статистика по областям без автономных округов до 2014 г. не приводится). Данные позже 2016 г. отсутствуют. Выделяем данные по диагнозам, которые могут быть связаны с социальным неблагополучием.

In [9]:
morbidity = pd.read_excel(
        'social_russia_data/morbidity_2005_2020_age_disease.xls',
        header=2, usecols='A:C, M:O')
morbidity.rename(
    {'Unnamed: 0': 'region', 'Unnamed: 1': 'type', 'Unnamed: 2': 'age'}, 
    axis=1, inplace=True)
morbidity = morbidity.dropna(subset=['2014', '2015', '2016'], how='all')
morbidity = federal_subj(morbidity, standard_names)

# Общая заболеваемость
mask = (
    (morbidity['type'] == 'Все заболевания')
    & (morbidity['age'] ==  'Всего')   
)
general_morbidity = morbidity[mask].copy()
general_morbidity.drop(columns=['type', 'age', '2014'], inplace=True)
general_morbidity.columns = pd.to_datetime(
    general_morbidity.columns).year

# Врожденные пороки развития
mask = (
    (morbidity['type'] == 'Врожденные аномалии (пороки развития), деформации и хромосомные нарушения')
    & (morbidity['age'] ==  'Всего')   
)
congenital_malformation = morbidity[mask].copy()
congenital_malformation.drop(columns=['type', 'age'], inplace=True)
congenital_malformation.columns = pd.to_datetime(
    congenital_malformation.columns).year

# Психические расстройства и расстройства поведения
mask = (
    (morbidity['type'] == 'Психические расстройства и расстройства поведения')
    & (morbidity['age'] ==  'Всего')   
)
psychosis = morbidity[mask].copy()
psychosis.drop(columns=['type', 'age'], inplace=True)
psychosis.columns = pd.to_datetime(
    psychosis.columns).year

# Подростковая беременность (до 14 лет)
mask = (
    (morbidity['type'] == 'Беременность, роды и послеродовой период')
    & (morbidity['age'] ==  '0-14 лет')   
)
teenage_pregnancy = morbidity[mask].copy()
teenage_pregnancy.drop(columns=['type', 'age'], inplace=True)
teenage_pregnancy.columns = pd.to_datetime(
    teenage_pregnancy.columns).year

display('Общая заболеваемость', general_morbidity.head(3))
display('Врожденные пороки развития', congenital_malformation.head(3))
display('Психические расстройства', psychosis.head(3))
display('Подростковая беременность', teenage_pregnancy.head(3))

'Общая заболеваемость'

,2015,2016
region,,
Алтайский край,109652.6,112764.7
Амурская область,83919.0,83412.0
Архангельская область без Ненецкого АО,101587.4,99842.1


'Врожденные пороки развития'

,2014,2015,2016
region,,,
Алтайский край,176.1,196.7,200.1
Амурская область,391.2,335.2,370.1
Архангельская область без Ненецкого АО,380.8,423.9,458.9


'Психические расстройства'

,2014,2015,2016
region,,,
Алтайский край,1454.0,1467.7,1272.1
Амурская область,512.4,485.3,446.7
Архангельская область без Ненецкого АО,646.0,693.5,685.3


'Подростковая беременность'

,2014,2015,2016
region,,,
Алтайский край,3.4,8.3,3.3
Амурская область,22.4,13.3,4.4
Архангельская область без Ненецкого АО,10.5,6.9,13.8


### 1.8. Рождаемость

Единица измерения - количество человек

In [10]:
newborn = pd.read_csv(
    'social_russia_data/newborn_2006_2022_monthly.csv', sep=';', decimal=',')
newborn.drop(columns='Unnamed: 198',inplace=True)
newborn.rename({'Region': 'region'}, axis=1, inplace=True)
names_to_drop = [
    'Архангельская область (кроме Ненецкого автономного округа)',
    'Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)',
    'Московская обл. в старых границах',
    'Москва в старых границах'
]
newborn = federal_subj(newborn, standard_names, names_to_drop=names_to_drop)
newborn.loc['Архангельская область без Ненецкого АО'] \
    = (newborn.loc['Архангельская область без Ненецкого АО']
      - newborn.loc['Ненецкий автономный округ'])
newborn.loc['Тюменская область без округов'] \
    = (newborn.loc['Тюменская область без округов']
       - newborn.loc['Ханты-Мансийский автономный округ - Югра']
       - newborn.loc['Ямало-Ненецкий автономный округ'])
    
# Суммируем данные за год
for year in range(2006, 2023):
    cols = []
    for col in newborn.columns:
        if str(year) in str(col):
            cols.append(col)
    newborn[year] = newborn[cols].sum(axis=1)
    newborn.drop(columns=cols, inplace=True)
newborn.drop(columns=2022, inplace=True)  # Неполный год
newborn = newborn.round(0).astype(int)
newborn.columns = pd.to_datetime(newborn.columns, format='%Y').year

newborn.head(3)

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
region,,,,,,,,,,,,,,,,
Алтайский край,25513,28209,30711,31038,30863,30739,32695,32461,31429,30065,28867,25525,23491,21117,19976,18992
Амурская область,10405,10935,11233,11384,11473,11188,11733,11483,11159,10781,10377,9430,8889,7919,7853,7343
Архангельская область без Ненецкого АО,13396,14438,14577,14776,14806,14253,14730,14584,14331,13908,13313,11714,10671,9662,8906,8487


### 1.9. Население

Единица измерения - количество человек

In [11]:
population = pd.read_excel(
        'social_russia_data/population.xlsx', header=1, usecols='A, C:Z')
population.rename({'Unnamed: 0': 'region'}, axis=1, inplace=True)

for i, val in enumerate(population['region']):
    if val == 'Раздел 1. Муниципальные образования субъектов Российской Федерации':
        population.loc[i, 'region'] = population.loc[i-1, 'region']
population.dropna(subset=population.columns[1:], how='all', inplace=True)

names_to_drop = [
        'Архангельская область (кроме Ненецкого автономного округа)',
        'Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)',
        'Муниципальные образования Алтайского края',
        'Алтайский муниципальный район',
        'Алтайский сельсовет',
        'Большекалтайский сельсовет',
        'Населенные пункты, входящие в состав муниципальных образований Алтайского края',
        'Кировский сельсовет'
]
population = federal_subj(population, standard_names, names_to_drop)
population.loc['Архангельская область без Ненецкого АО'] \
    = (population.loc['Архангельская область без Ненецкого АО']
      - population.loc['Ненецкий автономный округ'])
population.loc['Тюменская область без округов'] \
    = (population.loc['Тюменская область без округов']
       - population.loc['Ханты-Мансийский автономный округ - Югра']
       - population.loc['Ямало-Ненецкий автономный округ'])

newcols = []
for col in population.columns:
    newcols.append(re.search(r'\d+', col).group(0))
population.columns = pd.to_datetime(newcols).year

population.head(3)

,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
region,,,,,,,,,,,,,,,,,,,,,
Алтайский край,2662738.0,2651628.0,2641079.0,2621050.0,2602595.0,2571987.0,2539430.0,2503510.0,2473024.0,2453455.0,...,2398751.0,2390638.0,2384812.0,2376774.0,2365680.0,2350080.0,2332813.0,2317153.0,2296353.0,2268179.0
Амурская область,949526.0,935607.0,923055.0,911381.0,901044.0,887781.0,874018.0,861056.0,850502.0,844290.0,...,816910.0,811274.0,809873.0,805689.0,801752.0,798424.0,793194.0,790044.0,781846.0,772525.0
Архангельская область без Ненецкого АО,1372631.0,1349160.0,1328187.0,1309552.0,1290956.0,1273668.0,1257312.0,1239924.0,1224813.0,1215264.0,...,1159506.0,1148760.0,1139950.0,1130240.0,1121813.0,1111031.0,1100290.0,1092424.0,1082662.0,1069782.0


### 1.10. Процент населения за чертой бедности (доходы ниже прожиточного минимума)

In [12]:
poverty_percent = pd.read_csv(
    'social_russia_data/poverty_percent_by_regions_1992_2020.csv')
poverty_percent = pd.pivot_table(
    data=poverty_percent, values='poverty_percent', 
    columns='year', index='region').reset_index()
names_to_drop = [
    '        Архангельская область',
    '        Тюменская область'
]
poverty_percent = federal_subj(poverty_percent, standard_names, names_to_drop)
poverty_percent.columns = pd.to_datetime(
    poverty_percent.columns, format='%Y').year

poverty_percent.head(3)

year,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
region,,,,,,,,,,,,,,,,,,,,,
Алтайский край,NaN,NaN,NaN,33.7,46.8,45.7,52.9,53.8,53.9,47.3,...,22.6,20.6,17.6,17.1,18.0,17.8,17.5,17.4,17.6,17.5
Амурская область,NaN,NaN,NaN,36.1,28.2,26.3,31.2,38.0,47.7,45.3,...,20.4,16.0,16.2,14.8,15.2,17.0,16.7,15.6,15.7,15.2
Архангельская область без Ненецкого АО,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,13.0,14.5,14.5,16.2,14.9,13.9,12.5,12.7,12.3


### 1.10. Распределение малоимущего населения по социально-демографическим группам

В 2019 и 2020 гг. данные частично не предоставлены (NaN или нули по всем категориям). Также имеются случаи, когда данные по одной из групп отсутствуют (дано 0%), что, скорее всего, является следствием халатного статистического подсчета, т.к. соседние годы показывают другие (ненулевые) цифры. Во всех этих случаях пропущенных или недостоверных данных будем брать для каждого конкретного региона данные по предшествующему году. 

In [13]:
def socdem_readout(file):
    df = pd.read_excel(file, header=2)
    df.rename({
        'Unnamed: 0': 'region',
        'Все население': 'all',
        'Дети в возрасте до 16 лет': 'children',
        'Население старше трудоспособного возраста': 'retired',
        'Население трудоспособного возраста': 'employable'
        }, axis=1, inplace=True)
    
    names_to_drop = [
        '        Архангельская область',
        '        Тюменская область'
    ]
    df = federal_subj(df, standard_names, names_to_drop)
    return df


poverty_socdem_2017 = socdem_readout(
    'social_russia_data/poverty_socdem_2017.xls')
poverty_socdem_2018 = socdem_readout(
    'social_russia_data/poverty_socdem_2018.xls')
poverty_socdem_2019 = socdem_readout(
    'social_russia_data/poverty_socdem_2019.xls')
poverty_socdem_2020 = socdem_readout(
    'social_russia_data/poverty_socdem_2020.xls')

poverty_socdem = poverty_socdem_2020.copy()

mask = (poverty_socdem[['children', 'retired', 'employable']]
        .isna().all(axis=1))
poverty_socdem[mask] = poverty_socdem_2019

mask = ((poverty_socdem[['children', 'retired', 'employable']] == 0)
        .any(axis=1))
poverty_socdem[mask] = poverty_socdem_2018

mask = ((poverty_socdem[['children', 'retired', 'employable']] == 0)
        .any(axis=1))
poverty_socdem[mask] = poverty_socdem_2017

poverty_socdem.head(3)

,all,children,retired,employable
region,,,,
Алтайский край,100,31.4,9.2,59.4
Амурская область,100,38.4,4.7,56.9
Архангельская область без Ненецкого АО,100,28.5,7.2,64.3


### 1.11. Региональное производство

Объем отгруженных товаров собственного производства или работ/услуг, выполненных собственными силами, по видам деятельности, в тысячах рублей

In [14]:
reg_prod = pd.read_csv(
    'social_russia_data/regional_production_2017_2020.csv')
names_to_drop = [
    '        Архангельская область', 
    '        Тюменская область']
reg_prod = federal_subj(reg_prod, standard_names, names_to_drop) 
reg_prod.columns.values[-4:] = pd.to_datetime(reg_prod.columns[-4:]).year
reg_prod_total = (reg_prod
                  .groupby('region')[[2017, 2018, 2019, 2020]].sum()) 
reg_prod_total.columns = pd.to_datetime(
    reg_prod_total.columns, format='%Y').year

display('Региональное производство, по видам деятельности', reg_prod.head(5))
display('Региональное производство, общий объем', reg_prod_total.head(3))

'Региональное производство, по видам деятельности'

,production_field,2017,2018,2019,2020
region,,,,,
Алтайский край,"ВОДОСНАБЖЕНИЕ; ВОДООТВЕДЕНИЕ, ОРГАНИЗАЦИЯ СБОР...",8068621.4,1.032303e+07,9813456.6,1.045441e+07
Алтайский край,ДОБЫЧА ПОЛЕЗНЫХ ИСКОПАЕМЫХ,6143029.4,5.023392e+06,7745963.3,9.146146e+06
Алтайский край,ОБРАБАТЫВАЮЩИЕ ПРОИЗВОДСТВА,307969929.4,3.228103e+08,357699013.3,3.618801e+08
Алтайский край,"ОБЕСПЕЧЕНИЕ ЭЛЕКТРИЧЕСКОЙ ЭНЕРГИЕЙ, ГАЗОМ И ПА...",45806615.7,4.743969e+07,43900541.6,4.474102e+07
Алтайский край,Промышленное производство (промышленность),367988195.7,3.855964e+08,419158974.7,4.262217e+08


'Региональное производство, общий объем'

,2017,2018,2019,2020
region,,,,
Алтайский край,735976391.6,7.711927e+08,838317949.5,8.524434e+08
Амурская область,268844147.6,2.687608e+08,340026685.8,4.045765e+08
Архангельская область без Ненецкого АО,709009752.2,6.064711e+08,533584941.4,6.641070e+08


### 1.12. Розничная торговля на душу населения

In [15]:
retail = pd.read_excel(
    'social_russia_data/retail_turnover_per_capita_2000_2021.xls', 
    header=2, usecols='A, P:X')
retail.rename({'Unnamed: 0': 'region'}, axis=1, inplace=True)
retail.drop(0, inplace=True)
names_to_drop = [
        '        Архангельская область',
        '            Архангельская область (без АО)',
        '        Тюменская область',
        '            Тюменская область (без АО)'
]
retail = federal_subj(retail, standard_names, names_to_drop)
retail.columns = pd.to_datetime(retail.columns).year

retail.head(3)

,2013,2014,2015,2016,2017,2018,2019,2020,2021
region,,,,,,,,,
Алтайский край,118096,128376,134925,137844,143873,150444,159514,153605,174321
Амурская область,145301,163781,182491,191523,202038,214688,231113,245233,276635
Архангельская область без Ненецкого АО,154177,176420,194345,202977,217332,229922,240155,250033,280050


### 1.13. Бюджеты регионов и расходы на социальную политику

По данным сайта https://budget.permkrai.ru/

В миллионах рублей. Доходы бюджетов расзделены на собственные доходы (income, средства, получаемые из налогов, акцизов, использования имущества) и дотации из бюджетов других уровней (subsidies, как правило это средства федерального бюджета).

In [16]:
income = pd.read_excel('social_russia_data/regional_budget_incomes_2020.xlsx', 
                        header=2, usecols='A, C, E, G, I, K, M, O')
income['income'] = income[income.columns[-7:-1]].sum(axis=1) / 1_000_000 
income['subsidies'] = income[income.columns[-2]] / 1_000_000  
income.drop(columns=income.columns[-9:-2], inplace=True)
income.rename({'Unnamed: 0': 'region'}, axis=1, inplace=True)
income = federal_subj(income, standard_names) 

welfare = pd.read_excel('social_russia_data/regional_budget_expenses_2020.xlsx', 
                        header=2, usecols='A, Q', decimal=',')
welfare.rename({'Unnamed: 0': 'region', 'Факт.7': 'welfare'}, 
               axis=1, inplace=True)
welfare['welfare'] = (welfare['welfare'].apply(
    lambda x: float(x.replace(',', '.').replace(' ', ''))))
welfare = federal_subj(welfare, standard_names)

display('Доходы бюджета, млн руб.', income.head(3))
display('Расходы на социальную политику, млн руб.', welfare.head(3))

'Доходы бюджета, млн руб.'

,income,subsidies
region,,
Алтайский край,56637.879730,74306.235370
Амурская область,51571.555319,34011.885547
Архангельская область без Ненецкого АО,52963.993010,42447.952872


'Расходы на социальную политику, млн руб.'

,welfare
region,
Алтайский край,46350.3
Амурская область,26104.8
Архангельская область без Ненецкого АО,29656.2


### 1.14. Преступность

По открытым данным Генпрокуратуры за 2020 г.: форма 4-ЕГС раздел 1 "Общие сведения о состоянии преступности". Берем данные по преступлениям, погибшим и тяжело пострадавшим на территории Российской Федерации.

*Предоставленные в проекте формы 4-ЕГС раздел 4 содержат сведения о преступлениях, совершенных отдельными категориями лиц, и не отражают полную статистику!*

In [17]:
file = 'social_russia_data/4-EGS_Razdel_1_122020.xls'

crimes = pd.read_excel(
    file, header=13, usecols='B, D, G, AB, AD')
crimes['registered crimes'] = crimes[2] - crimes[5]
crimes['killed'] = crimes[26]
crimes['grievous harm'] = crimes[28]
crimes.drop([2, 5, 26, 28], axis=1, inplace=True)
crimes.rename({'Unnamed: 1': 'region'}, axis=1, inplace=True)

felony = pd.read_excel(
    file, sheet_name=1, header=13, usecols='D, G')
felony['felonies'] = felony[2] - felony[5]

misdemeanor = pd.read_excel(
    file, sheet_name=2, header=13, usecols='D, G')
misdemeanor['misdemeanors'] = misdemeanor[2] - misdemeanor[5]

crimes = pd.concat([
    crimes[['region', 'registered crimes']],
    felony[['felonies']],
    misdemeanor[['misdemeanors']],
    crimes[['killed', 'grievous harm']]
    ], axis=1
)

crimes = federal_subj(crimes, standard_names)
crimes.head(3)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


,registered crimes,felonies,misdemeanors,killed,grievous harm
region,,,,,
Алтайский край,39024,1529,8887,411,647
Амурская область,18420,560,4967,223,255
Архангельская область без Ненецкого АО,19074,823,4569,166,288


### 1.15. Дети

Данные портала https://tochno.st за 2020 г.

In [18]:
children = pd.read_excel(
    'social_russia_data/report_orphanhood_2020.xlsx',
    header=1, usecols='B:E')
children.rename({
    'Регион': 'region',
    'Всего детей в регионе, человек': 'children',
    'Нуждаются в устройстве в семьи, человек': 'orphanage',
    'Устройство детей в семьи, %': 'adopted %'
    }, axis=1, inplace=True)
children['adopted %'] = (children['adopted %']
    .mask(children['adopted %'] == 'мало данных', np.NaN)).astype(float)
children = federal_subj(children, standard_names)
children.head(3)

,children,orphanage,adopted %
region,,,
Алтайский край,490149,983,59.0
Амурская область,178702,676,65.0
Архангельская область без Ненецкого АО,226228,785,60.0


### 1.16. Пенсионеры

Данные портала https://tochno.st

In [19]:
retired = pd.read_csv(
    'social_russia_data/problem_ageing_all.csv', decimal=',')
cols = [
    'Название показателя', 'Ед.измерения', 'Название региона', 'Сегмент',
    '2015', '2016', '2017', '2018', '2019', '2020', '2021'
]
retired = retired[cols]
retired.replace('н/д', np.NaN, inplace=True)
retired.replace('#Н/Д', np.NaN, inplace=True)
retired.columns.values[-7:] = pd.to_datetime(retired.columns[-7:]).year
retired.rename({'Название региона': 'region'}, axis=1, inplace=True)

# Пенсии
mask = retired['Название показателя'] == 'Средний размер пенсии'
pension = retired[mask].copy()
pension.drop(columns=['Название показателя', 'Ед.измерения', 'Сегмент', 2021], 
             inplace=True) 
for year in range(2015, 2021):
    pension[year] = (pension[year]
                     .apply(lambda x: float(str(x).replace(',','.'))))
pension = pension.groupby('region').sum().reset_index()  # Страх. + накопит.
pension = federal_subj(pension, standard_names)

# Число пенсионеров
mask = ((retired['Название показателя'] == 'Люди в возрасте старше трудоспособного')
        & (retired['Ед.измерения'] == 'человек') & (retired['Сегмент'] == 'всего'))
retired_number = retired[mask].copy()
retired_number.drop(columns=['Название показателя', 'Ед.измерения', 'Сегмент'],
                     inplace=True) 
for year in range(2015, 2022):
    retired_number[year] = (retired_number[year]
                     .apply(lambda x: float(str(x).replace(',','.'))))
retired_number = federal_subj(retired_number, standard_names)

# Процент пенсионеров
mask = ((retired['Название показателя'] == 'Люди в возрасте старше трудоспособного')
        & (retired['Ед.измерения'] == '%') & (retired['Сегмент'] == 'всего'))
retired_percent = retired[mask].copy()
retired_percent.drop(columns=['Название показателя', 'Ед.измерения', 'Сегмент'],
                     inplace=True) 
for year in range(2015, 2022):
    retired_percent[year] = (retired_percent[year]
                     .apply(lambda x: float(str(x).replace(',','.'))))
retired_percent = federal_subj(retired_percent, standard_names)

display('Средний размер пенсии', pension.head(3))
display('Число пенсионеров', retired_number.head(3))
display('Процент пенсионеров', retired_percent.head(3))

'Средний размер пенсии'

,2015,2016,2017,2018,2019,2020
region,,,,,,
Алтайский край,12198.70,12523.82,13147.23,14376.00,15243.87,16128.19
Амурская область,13698.68,14092.91,14780.44,16065.37,17005.85,17998.70
Архангельская область без Ненецкого АО,0.00,0.00,0.00,0.00,0.00,21753.88


'Число пенсионеров'

,2015,2016,2017,2018,2019,2020,2021
region,,,,,,,
Алтайский край,604333.0,618276.0,629725.0,639803.0,647643.0,621984.0,622583.0
Амурская область,175375.0,178772.0,181429.0,183228.0,185016.0,176633.0,176179.0
Архангельская область без Ненецкого АО,283394.0,288882.0,293477.0,297292.0,300715.0,288908.0,290570.0


'Процент пенсионеров'

,2015,2016,2017,2018,2019,2020,2021
region,,,,,,,
Алтайский край,25.3,26.0,26.6,27.2,27.8,26.8,27.1
Амурская область,21.7,22.2,22.6,22.9,23.3,22.4,22.5
Архангельская область без Ненецкого АО,33.0,25.6,26.2,26.8,27.3,26.4,26.8


### 1.17. Прожиточный минимум.

По данным Росстата. Берем данные на 4 квартал 2020 г. для трудоспособного населения.

In [20]:
living_wage = pd.read_excel(
    'social_russia_data/living_wage_2020.xls',
    header=4, usecols='A, I')
living_wage.rename({
    'Unnamed: 0': 'region',
    'Трудоспособное население.3': 'living wage',
    }, axis=1, inplace=True)
names_to_drop = ['        Архангельская область',
                 '        Тюменская область']
living_wage = federal_subj(living_wage, standard_names, names_to_drop)
living_wage.head(3)

,living wage
region,
Алтайский край,10904.0
Амурская область,14331.0
Архангельская область без Ненецкого АО,14507.0


### 1.18. Сводная таблица полученных данных. Формирование датасета проекта.

Выделены признаки, которые потенциально могут быть использованы для кластерного анализа.

| Dataframe | Описание | Единица измерения | Период |
| - | - | - | - |
| ***per_capita:*** | Средний денежный доход (в месяц) | руб. / чел. | 2015-2020 |
| real_incomes | Реальные денежные доходы (покупательная способность: с учетом роста цен) | процент к предыдущему году | 2015-2020 |
| ***formal_wage*** |Среднемесячная номинальная начисленная заработная плата | руб. / чел. | 2015-2020 |
| real_pay | Реальная начисленная заработная плата (покупательная способность: с учетом роста цен) | процент к предыдущему году | 2015-2020 |
| child_mortality | Детская смертность (до достижения первого года жизни) | чел. | 1990-2021 |
| disabled_18_60 | Инвалидность среди трудоспособного возраста (18-60 лет) | чел. | 2017-2022 |
| alco, drug | Кол-во алкогольных и наркотических психозов | кол-во. / 100 тыс. населения | 2005-2018 |
| ***grp*** | Валовый региональный продукт (объем произведенных товаров и услуг) | руб. / чел. | 2011-2020 |
| ***housing*** | Жилищные условия: средняя жилая площадь, плохое состояние жилья, отсутствие средств | кв.м., %, % | 2020 |
| general_morbidity, congenital_malformation, psychosis, teenage_pregnancy | Общая заболеваемость, врожденные пороки развития, психические расстройства, подростковая беременность | кол-во. / 100 тыс. населения | 2015-2016 |
| newborn | Рождаемость | чел. | 2006-2021 |
| population | Население | чел. | 1999-2022 |
| ***poverty_percent*** | Процент населения за чертой бедности | %  | 1995-2020 |
| poverty_socdem_2017 - 2020 | Распределение малоимущих по социально-демографическим группам, в 2019-2020 гг. данные частично отсутствуют| % | 2017-2020 |
| reg_prod, ***reg_prod_total*** | Региональное производство по видам деятельности и общее | тыс. руб. | 2017-2020 |
| ***retail*** | Розничная торговля | руб. / чел. | 2013-2021 |
| ***income*** | Доходы бюджета | млн руб. | 2020 |
| welfare | Расходы на социальную политику | млн руб. | 2020 |
| crime | Зарегистрированные преступления, тяжкие и особо тяжкие преступления, количество погибших и тяжело пострадавших | Кол-во | 2020 |
| children | Количество детей, дети-сироты | чел. | 2020 |
| pension | Средний размер пенсии | руб. / чел. | 2015-2020 |
| retired_number, retired_percent | Процент пенсионеров | чел., % | 2015-2021 |
| ***living_wage*** | Прожиточный минимум для трудоспособного населения на 4-й квартал | руб. / чел.| 2020 |

Собираем датасет с данными по 2020 г. (преимущественно). Частично нормируем значения для удобства последующего сравнения (процент населения, показатель на душу населения, показатель на 100 тыс. населения). 

Описание финального датасета:
| Признак | Описание |
| - | - |
| per capita | среднемесячный денежный доход на человека, в рублях|
| formal wage | среднемесячная номинальная начисленная заработная плата на человека, в рублях|
| child mortality | детская смертность, в процентах от количества новорожденных в 2020 г.|
| disabled 18_60 | инвалиды в возрасте 18-60 лет в процентах к населению соответствующего возраста (оценочно)|
| alco (2018) | количество впервые зарегистрированных алкогольных псизозов на 100 тыс. населения, данные 2018 г.|
| drug (2018) | количество впервые зарегистрированных наркотических псизозов на 100 тыс. населения, данные 2018 г.|
| grp | валовый региональный продукт (объем произведенных товаров и услуг) на душу населения, в рублях|
| living area | средняя жилая площадь на человека|
| bad conditions | процент респондентов, указавших на плохое состояние жилья|
| no funds | процент респондентов, рассчитывающих на получение жилья от государства|
| morbidity (2016) | общая заболеваемость на 100 тыс. населения, данные 2016 г.|
| congenital malformation(2016) | врожденные пороки развития на 100 тыс. населения, данные 2016 г.|
| psychosis (2016) | психические расстройства на 100 тыс. населения, данные 2016 г.|
| teenage pregnancy (2016) | подростковая беременность на 100 тыс. населения, данные 2016 г.|
| birth rate | рождаемость на 100 тыс. населения|
| population | население, количество человек|
| poverty | процент населения за чертой бедности|
| the poor: children | процент детей среди малообеспеченного населения|
| the poor: retired | процент пенсионеров среди малообеспеченного населения|
| the poor: employable | процент людей трудоспособного возраста среди малообеспеченного населения|
| regional production | региональное производство по всем видам деятельности на душу населения, в рублях|
| retail | розничная торговля на душу населения, в рублях|
| income | собственные доходы регионального бюджета на душу населения, в рублях|
| subsidies | дотации федерального бюджета на душу населения, в рублях|
| welfare | расходы регионального бюджета на социальную политику, в рублях на душу населения|
| registered crimes | общее количество зарегистрированных преступлений|
| felonies | количество особо тяжких преступлений|
| misdemeanors | количество тяжких преступлений|
| killed | количество погибших в результате противоправных действий|
| grievous harm | количество тяжело пострадавших в результате противоправных действий|
| children % | процент детей среди всего населения|
| orphanage | количество детей в детских домах|
| adopted % | процент усыновленных/удочеренных детей|
| pension | средний размер пенсии, в рублях|
| retired % | процент пенсионеров среди всего населения|
| living wage| прожиточный минимум, в рублях|

In [21]:
russia_regions_2020 = pd.concat([
    per_capita[[2020]].rename({2020: 'per capita'}, axis=1),
    formal_wage[[2020]].rename({2020: 'formal wage'}, axis=1),
    (pd.DataFrame(child_mortality[2020] / newborn[2020] * 100)
        .rename({2020: 'child mortality'}, axis=1)),
    (pd.DataFrame(
        disabled_18_60[2020] 
        / (population[2020]-children['children']-retired_number[2020]) * 100)
        .rename({0: 'disabled 18_60'}, axis=1)),
    alco[[2018]].rename({2018: 'alco (2018)'}, axis=1),
    drug[[2018]].rename({2018: 'drug (2018)'}, axis=1), 
    grp[[2020]].rename({2020: 'grp'}, axis=1),  
    housing,
    general_morbidity[[2016]].rename({2016: 'morbidity (2016)'}, axis=1),
    (congenital_malformation[[2016]]
        .rename({2016: 'congenital malformation (2016)'}, axis=1)),
    psychosis[[2016]].rename({2016: 'psychosis (2016)'}, axis=1),
    teenage_pregnancy[[2016]].rename({2016: 'teenage pregnancy (2016)'}, 
                                     axis=1),
    (pd.DataFrame(newborn[2020] / population[2020] * 100_000)
        .rename({2020: 'birth rate'}, axis=1)),
    population[[2020]].rename({2020: 'population'}, axis=1),
    poverty_percent[[2020]].rename({2020: 'poverty'}, axis=1),
    poverty_socdem.drop(columns='all').rename({
        'children': 'the poor: children',
        'retired': 'the poor: retired',
        'employable': 'the poor: employable',
        }, axis=1),
    (pd.DataFrame(reg_prod_total[2020] / population[2020] * 1000)
        .rename({2020: 'regional production'}, axis=1)),
    retail[[2020]].rename({2020: 'retail'}, axis=1),
    (pd.DataFrame(income['income'] / population[2020] * 1_000_000)
        .rename({0: 'income'}, axis=1)),
    (pd.DataFrame(income['subsidies'] / population[2020] * 1_000_000)
        .rename({0: 'subsidies'}, axis=1)),
    (pd.DataFrame(welfare['welfare'] / population[2020] * 1_000_000)
        .rename({0: 'welfare'}, axis=1)),
    crimes,
    (pd.DataFrame(children['children'] / population[2020] * 100)
        .rename({0: 'children %'}, axis=1)),
    children[['orphanage', 'adopted %']],
    pension[[2020]].rename({2020: 'pension'}, axis=1), 
    retired_percent[[2020]].rename({2020: 'retired %'}, axis=1), 
    living_wage   
    ], 
    axis=1
)

russia_regions_2020.to_csv('social_russia_data/russia_regions_2020.csv')
russia_regions_2020

,per capita,formal wage,child mortality,disabled 18_60,alco (2018),drug (2018),grp,living area,bad conditions,no funds,...,felonies,misdemeanors,killed,grievous harm,children %,orphanage,adopted %,pension,retired %,living wage
region,,,,,,,,,,,,,,,,,,,,,
Алтайский край,23864.0,30072.0,0.485583,5.093525,88.1,17.4,291156.9,18.0,0.3,0.8642,...,1529,8887,411,647,21.153070,983,59.0,16128.19,26.8,10904.0
Амурская область,35499.0,52430.0,0.534827,5.766846,113.3,24.4,571362.1,17.3,1.3,1.0731,...,560,4967,223,255,22.619247,676,65.0,17998.70,22.4,14331.0
Архангельская область без Ненецкого АО,34852.0,52779.0,0.325623,4.674270,64.7,9.2,514200.4,16.9,1.4,1.2998,...,823,4569,166,288,20.708809,785,60.0,21753.88,26.4,14507.0
Астраханская область,25199.0,38885.0,0.586564,3.632293,22.2,0.2,526950.9,19.2,0.8,2.9028,...,736,2818,93,169,22.620409,376,66.0,15427.85,23.9,11287.0
Белгородская область,32841.0,37442.0,0.380505,7.250936,41.3,3.2,646569.0,17.8,0.7,0.3840,...,628,3208,159,249,18.764730,219,82.0,17438.93,27.2,10403.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Чеченская Республика,24596.0,29771.0,0.601109,17.854091,4.1,0.6,171029.5,19.2,0.6,0.5539,...,55,717,24,21,37.089495,0,100.0,16165.00,10.3,11572.0
Чувашская Республика,21155.0,31844.0,0.212258,4.385201,79.0,1.2,287210.3,14.2,1.2,1.8768,...,1097,3418,141,188,21.259663,161,77.0,16111.97,24.9,10414.0
Чукотский автономный округ,89541.0,120641.0,1.489758,3.515586,208.7,0.0,2404271.2,14.1,1.9,0.0000,...,24,167,28,20,25.099427,60,69.0,30144.56,14.5,24711.0


### Вывод

Проведено считывание экономических и демографических данных из предоставленных файлов различного формата и из дополнительных источников. Результаты сохраняются в датасеты со стандартизованными названиями федеральных субъектов РФ (85 субъектов). На основе полученных датасетов сформирована финальная таблица по срезу 2020 г., которая используется далее в проекте. Помимо этого, структура ноутбука позволяет с помощью минимальных изменений кода оперативно комбинировать различные наборы характеристик (срезы по другим годам, динамику, дополнительные признаки) для возможного анализа финасово-экономических, демографических, медицинских, криминальных показателей по российским регионам.  